In [1]:
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest.trainer import Trainer

In [4]:
dims = 3

In [5]:
test_data = np.random.normal(size=(10, dims))
test_data = torch.from_numpy(test_data).float()

In [6]:
trainers = []
trainers.append(Trainer(dims, 64,  num_blocks=2, num_layers=1, flow='nvp'))
trainers.append(Trainer(dims, 64,  num_blocks=1, flow='spline'))

[nnest.trainer] [INFO] SingleSpeedNVP(
  (flow): NormalizingFlow(
    (flows): ModuleList(
      (0): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=3, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=3, bias=True)
        )
        (translate_net): Sequential(
          (0): Linear(in_features=3, out_features=64, bias=True)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): ReLU()
          (4): Linear(in_features=64, out_features=3, bias=True)
        )
      )
      (1): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=3, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=3, bias=True)
      

In [7]:
for t in trainers:
    z, log_det = t.netG.forward(test_data)
    print(z.shape, log_det.shape)
    x, log_det = t.netG.inverse(z)
    print(z.shape, log_det.shape)
    print(torch.max(x - test_data))
    log_probs = t.netG.log_probs(test_data)
    print(log_probs.shape)
    samples = t.netG.sample(10)
    print(samples.shape)

tensor([[-1.8343,  0.6900, -1.4740],
        [-0.0091, -0.5250, -0.3054],
        [-0.6794,  0.2499,  0.4996],
        [-0.2760,  0.6545, -0.3609],
        [ 0.3404,  0.6093, -1.6958],
        [-1.2276, -0.3060, -0.3729],
        [-1.5211,  1.3302, -1.9556],
        [ 1.6214, -0.0992, -0.0880],
        [ 0.2185,  1.4612,  0.7142],
        [-0.0150,  0.4572, -0.4496]])
tensor([0., 1., 0.])
tensor([[-1.7745,  0.6900, -1.3806],
        [-0.0732, -0.5250, -0.3471],
        [-0.7393,  0.2499,  0.4646],
        [-0.3954,  0.6545, -0.3331],
        [ 0.1560,  0.6093, -1.5882],
        [-1.2530, -0.3060, -0.3922],
        [-1.5230,  1.3302, -1.8501],
        [ 1.4432, -0.0992, -0.1042],
        [-0.0586,  1.4612,  0.6929],
        [-0.1457,  0.4572, -0.4170]], grad_fn=<AddBackward0>)
tensor([1., 0., 1.])
torch.Size([10, 3]) torch.Size([10])
torch.Size([10, 3]) torch.Size([10])
tensor(1.1921e-07, grad_fn=<MaxBackward1>)
tensor([[-1.8343,  0.6900, -1.4740],
        [-0.0091, -0.5250, -0.3054],
 

In [8]:
num_slow = 2
num_fast = 3
dims = num_slow + num_fast

In [9]:
test_data = np.random.normal(size=(10, dims))
test_data = torch.from_numpy(test_data).float()

In [10]:
trainers = []
trainers.append(Trainer(dims, 64,  num_slow=num_slow, num_blocks=1, num_layers=1, flow='nvp'))
#trainers.append(Trainer(dims, 64,  num_slow=num_slow, num_blocks=3, flow='spline'))

tensor([0., 1., 0.])
tensor([0., 1.])
[nnest.trainer] [INFO] FastSlowNVP(
  (fast_flow): NormalizingFlow(
    (flows): ModuleList(
      (0): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Tanh()
          (4): Linear(in_features=64, out_features=2, bias=True)
        )
        (translate_net): Sequential(
          (0): Linear(in_features=2, out_features=64, bias=True)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): ReLU()
          (4): Linear(in_features=64, out_features=2, bias=True)
        )
      )
    )
  )
  (slow_flow): NormalizingFlow(
    (flows): ModuleList(
      (0): CouplingLayer(
        (scale_net): Sequential(
          (0): Linear(in_features=3, out_features=64, bias=True)
          (1): Tanh()
          (2): Linear(in_features=64, out_features=6

In [11]:
for t in trainers:
    z, log_det = t.netG.forward(test_data)
    print(z.shape, log_det.shape)
    x, log_det = t.netG.inverse(z)
    print(z.shape, log_det.shape)
    print(torch.max(x - test_data))
    log_probs = t.netG.log_probs(test_data)
    print(log_probs.shape)
    samples = t.netG.sample(10)
    print(samples.shape)
    dz = torch.randn_like(z) * 0.01
    dz[:, 0:num_slow] = 0.0
    xp, log_det = t.netG.inverse(z+dz)
    print(torch.max((x-xp)[:, 0:num_slow])) 

tensor([[-0.0046, -0.0734, -0.7463,  1.4833, -1.4420],
        [ 0.8642,  0.0568,  0.4960, -0.1731, -0.5101],
        [ 0.3077,  0.8890, -1.2339,  1.1464,  0.2806],
        [ 1.0952, -0.0396,  1.4894, -0.8415, -1.7065],
        [-0.4499,  2.1340,  0.2500, -1.2263,  1.2572],
        [ 0.2208,  2.9792,  2.5698,  1.3756, -0.4611],
        [-1.0325, -1.3342,  1.8551, -1.8485,  0.1847],
        [-0.2020, -0.4421,  0.1139,  0.4386, -0.9212],
        [ 0.0068,  1.4043, -0.8407, -0.0283,  0.7006],
        [ 2.2954,  0.7273,  0.7321, -0.4663,  0.7272]])
2
tensor([[-0.0046, -0.0734],
        [ 0.8642,  0.0568],
        [ 0.3077,  0.8890],
        [ 1.0952, -0.0396],
        [-0.4499,  2.1340],
        [ 0.2208,  2.9792],
        [-1.0325, -1.3342],
        [-0.2020, -0.4421],
        [ 0.0068,  1.4043],
        [ 2.2954,  0.7273]])
tensor([[-0.0046, -0.0734],
        [ 0.8642,  0.0568],
        [ 0.3077,  0.8890],
        [ 1.0952, -0.0396],
        [-0.4499,  2.1340],
        [ 0.2208,  2.9792]

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1